In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from pathlib import Path
from plaid.compression.hourglass_lightning import HourglassTransformerLightningModule
import os

# dirpath = Path("/homefs/home/lux70/storage/plaid/checkpoints/hourglass/2024-02-19T11-10-18")
dirpath = Path("/homefs/home/lux70/storage/plaid/checkpoints/hourglass/2024-02-20T03-24-07")
list(dirpath.glob("*"))
ckpt = torch.load(dirpath / "last.ckpt")

In [3]:
# need to this this explicitly because didn't call save hyperparameter during training, now fixed for future versions.

# electric-snow
model = HourglassTransformerLightningModule(
    dim=1024,
    depth=[4,2,4],
    dim_head=64,
    downproj_factor=4,
    shorten_factor=2
)

# astral-sponge
# model = HourglassTransformerLightningModule(
#     dim=1024,
#     depth=[4,2,4],
#     dim_head=64,
#     # downproj_factor=4,
#     shorten_factor=2
# )

device = torch.device("cuda")
model.load_state_dict(ckpt['state_dict'])

<All keys matched successfully>

In [4]:
# from evo.dataset import FastaDataset
# # dataset = FastaDataset("/homefs/home/lux70/storage/data/rocklin/rocklin_stable.fasta")
# dataset = FastaDataset("/homefs/home/lux70/storage/data/cath/cath-dataset-nonredundant-S40.atom.fa")
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)

# from plaid.transforms import get_random_sequence_crop_batch

# headers, sequences = next(iter(dataloader))
# sequences = get_random_sequence_crop_batch(sequences, max_len=512)
# print(sequences[:3])
# print(headers[:3])

In [5]:
from plaid.datasets import CATHStructureDataModule

shard_dir = "/homefs/home/lux70/storage/data/cath/shards/"
pdb_dir = "/data/bucket/lux70/data/cath/dompdb"

dm = CATHStructureDataModule(
    shard_dir,
    pdb_dir,
    seq_len=256,
    batch_size=4,
    num_workers=2,
    path_to_filtered_ids_list="/homefs/home/lux70/accurate_cath_ids.txt"
) 
    
dm.setup()
train_dataloader = dm.train_dataloader()
batch = next(iter(train_dataloader))
print(len(train_dataloader.dataset))

Did not find 4001 IDs, including ['4oe6B02', '3en9A02', '1tt5C01']
Did not find 16140 IDs, including ['4rx6D00', '1bw0A02', '3qokA01']


/homefs/home/lux70/miniconda3/envs/plaid/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


16140


In [ ]:
x = batch[0].to(device)
sequences = batch[1]
x = x.to(device)

In [ ]:
from plaid.esmfold.misc import batch_encode_sequences
_, mask, _, _, _ = batch_encode_sequences(sequences)
mask = mask.to(device)
model = model.to(device)

In [ ]:
from plaid.utils import LatentScaler
latent_scaler = LatentScaler()
x = latent_scaler.scale(x)

In [ ]:
recons = model(x, mask.bool())
print(torch.mean((recons - x) ** 2))

In [ ]:
print(recons[:2, :2, :2])
print(x[:2, :2,:2])


In [ ]:
from plaid.proteins import LatentToStructure

structure_constructor = LatentToStructure()

In [ ]:
structure_constructor.to(device)

In [ ]:
recons_struct = structure_constructor.to_structure(recons, sequences)

In [ ]:
orig_struct = structure_constructor.to_structure(x, sequences)

In [ ]:
for i, pdbstr in enumerate(recons_struct[0]):
    with open(f"recons_pred_{i}.pdb", "w") as f: 
        f.write(pdbstr)

In [ ]:
for i, pdbstr in enumerate(orig_struct[0]):
    with open(f"orig_pred_{i}.pdb", "w") as f: 
        f.write(pdbstr)

In [ ]:
import glob
recons_pdbs = glob.glob("recons_*.pdb")
orig_pdbs = glob.glob("orig_*.pdb")
print(orig_pdbs)

In [ ]:
from plaid.utils import run_tmalign

for orig, recons in zip(orig_pdbs, recons_pdbs):
    print(run_tmalign(orig, recons))

In [ ]:
from plaid.utils import view_py3Dmol

for i in range(4):
    view_py3Dmol(orig_struct[0][i])

In [ ]:
recons_pdbs[0]